In [ ]:
pip install mpld3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import matplotlib.pyplot as plt, mpld3
mpld3.enable_notebook()

%matplotlib inline

In [ ]:
# Calibration for x axis

m = 0.06275314836002217
b = -0.19536577738644514

In [ ]:
# Utility functions

from scipy.optimize import curve_fit

def gauss(x, A, x0, std_dev):
    return A * np.exp(-(x - x0) ** 2 / (2 * std_dev ** 2))

def gauss_fit(x, y):
    peak_index = np.argmax(y)
    peak_value = x[peak_index]
    poissonError = np.sqrt(y)
    std_dev = np.sqrt(sum((x - peak_index) ** 2) / (sum(y)-1)) #sample standard deviation
    
    #print(max(y), peak_index, std_dev)
    popt, pcov = curve_fit(gauss, x, y, p0=[ max(y), peak_index, std_dev], sigma = poissonError, maxfev=5000)
    return popt, pcov

def normal_gauss(x,A,x0,std_dev): 
    return A/np.sqrt(2*np.pi*std_dev**2)*np.exp(-(x-x0)**2/(2*std_dev**2))

def gauss_fit_to_spectrum(input_x,input_y,x1,x2, angle):
    # Select peak to fit
    xdata = input_x[x1:x2]
    ydata = input_y[x1:x2]
    [A, x0, std_dev],pcov = gauss_fit(xdata, ydata)
    # offset, amplitude, center, sigma
    
    [A_err, x0_err, std_dev_err] = np.sqrt(np.diag(pcov))
    print( [A_err, x0_err, std_dev_err])
    # Plot input spectra with correct scaling
    plt.plot(input_x*m+b,input_y, label='Subtracted Spectra')
    
    # Plot selected peak to fit
    plt.plot(xdata*m+b, ydata, label='Selected Peak')
    
    # Plot fitted gaussian to peak
    plt.plot(input_x*m+b, gauss(input_x, A, x0, std_dev), '--r', label='Fit')
#     plt.annotate(f'Mean={round(x0*m+b,2)}keV, $\sigma$={round(sigma,2)}, Amplitude={round(A,2)}counts/s',(0.2,0.5), xycoords='figure fraction')
    plt.legend()
    plt.title(angle + ' deg')
    plt.xlabel('Energy Levels (KeV)')
    plt.ylabel('Counts/s')
    plt.show()
    return np.array([x0*m+b,A,std_dev,x0_err,A_err,std_dev_err])

# Background

In [ ]:
#%matplotlib qt
plt.rcParams.update({'font.size': 20})
plt.rcParams["font.family"] = "serif"
plt.title('Americium on Germanium Detector Background')
plt.xlabel('Energies (keV)')
plt.ylabel('Intensity (counts/s)')
background =np.loadtxt('Calibrations/BackgroundRadiationGe85gain1500volts840sec.Spe',skiprows=12, max_rows=1023)
plt.plot(background)

In [ ]:
background

# Aluminum

In [ ]:
# Read files

files = glob.glob('Americium_Aluminum/*.Spe')
am_al_dict = {}

for f in files:
    angle = f.split('AmericiumAl')[1].split('degrees')[0]
    time = f.split('volts')[1].split('sec')[0]
    data = np.loadtxt(f,skiprows=12, max_rows=1023)
    entry = {'filename':f, 'time':time,'data':data}
    am_al_dict[angle] = entry

In [ ]:
# Define variables

bins = np.linspace(1,1023,1023)
plt.rcParams['figure.figsize'] = [10, 5]
data = []

In [ ]:
# Select compton energy peaks

am_al_dict['30']['x1'] = 870
am_al_dict['30']['x2'] = 937

am_al_dict['50']['x1'] = 865
am_al_dict['50']['x2'] = 935

am_al_dict['70']['x1'] = 830
am_al_dict['70']['x2'] = 935

am_al_dict['90']['x1'] = 750
am_al_dict['90']['x2'] = 900

am_al_dict['110']['x1'] = 770
am_al_dict['110']['x2'] = 870

am_al_dict['130']['x1'] = 750
am_al_dict['130']['x2'] = 850

# am_al_dict['150']['x1'] = 650
# am_al_dict['150']['x2'] = 800

In [ ]:
bins = np.linspace(1,1023,1023)
plt.rcParams['figure.figsize'] = [10, 5]

#%matplotlib qt
for angle in ['30','50','70','90','110','130']:
    entry = am_al_dict[angle]
    plt.figure(figsize=(10,5))
    # plt.plot(entry['data'], label = angle)

    x1 = entry['x1']
    x2 = entry['x2']
    
    # Use average background
    bg_level = np.mean(entry['data'][200:600])/int(entry['time'])
    #bg_level_poisson_error = np.sqrt(entry['data'][200:600])/int(entry['time'])
    
    # Background subtracted spectrum
    #spectrum = entry['data']/int(entry['time'])-bg_level
    # Plot background level
    plt.axhline(bg_level,linestyle='dashed')
    
    plt.errorbar(bins*m+b, entry['data'], yerr=np.sqrt(entry['data']),fmt='o',label='data') 
    

In [ ]:
#%matplotlib qt

In [ ]:
# def gauss_fit(x, y):
#     mean = sum(x * y) / sum(y)
#     sigma = np.sqrt(sum(y * (x - mean) ** 2) / sum(y))
#     err = np.sqrt(y)
#     popt, pcov = curve_fit(normal_gauss, x, y, sigma=err, absolute_sigma=True, p0=[ max(y), mean, sigma])
#     return popt, pcov
bins = np.linspace(1,1023,1023)

entry = am_al_dict['50']

x1 = entry['x1']
x2 = entry['x2']
    
input_x = bins*m+b
input_y = entry['data']
yerr = np.zeros(len(input_y))
yerr[np.where(input_y != 0.)] = np.sqrt(input_y[np.where(input_y != 0.)])

#bg_level_err = np.sqrt(entry['data'][200:600])/int(entry['time'])


#total_err = np.sqrt(np.square(yerr) + np.square(bg_level)) #background and spectra error added in quadrature. the error on the background and spectra is poisson distributed. so the error is the sqrt of the mean (or y) 

plt.figure()
plt.errorbar(input_x, input_y , yerr=yerr,fmt='o',label='data')

xdata = input_x[x1:x2]
ydata = input_y[x1:x2]
err = yerr[x1:x2]


#print(len(bg_level_err))
#print(len(xdata))



mean = sum(xdata * ydata) / sum(ydata)
sigma = (max(xdata) - min(xdata)) / 2
plt.errorbar(xdata, ydata , yerr=err,fmt='o',label='selected peak')


[A, x0, sigma],pcov = curve_fit(normal_gauss, xdata, ydata, sigma=err,absolute_sigma=True, p0=[ max(ydata), mean, sigma])

# offset, amplitude, center, sigma

[A_err, x0_err, sigma_err] = np.sqrt(np.diag(pcov))
print( [A_err, x0_err, sigma_err],[A, x0, sigma])

plt.plot(input_x, normal_gauss(input_x, A, x0, sigma), '--r', label='fit')

plt.title('Gaussian fit ' + angle + ' deg')
plt.xlabel('Energy Levels (KeV)')
plt.ylabel('Counts')
plt.legend()


In [ ]:
np.sum(normal_gauss(input_x, A, x0, sigma))

In [ ]:
bins = np.linspace(1,1023,1023)
plt.rcParams['figure.figsize'] = [10, 5]
al_data = []

%matplotlib inline
for angle in ['30','50','70','90','110','130']:
    entry = am_al_dict[angle]
    plt.figure(figsize=(10,5))
    # plt.plot(entry['data'], label = angle)

    x1 = entry['x1']
    x2 = entry['x2']
    # Use average background
    bg_level = np.mean(entry['data'][200:600])/int(entry['time'])
    #bg_level_poisson_error = np.sqrt(entry['data'][200:600])/int(entry['time'])
    
    # Background subtracted spectrum
    spectrum = entry['data']
    #/int(entry['time'])-bg_level

    # Plot background level
    plt.axhline(bg_level,linestyle='dashed')
    
    # Intensity Ratio
#     I_ratio = np.sum(entry['data'][x1:x2])/ np.sum(entry['data'])
    
    # Fit peak and plot data
    al_data.append(np.append(np.array([int(angle),int(am_al_dict[angle]['time'])]),gauss_fit_to_spectrum(bins*m+b,spectrum,x1,x2,angle)))
#     al_data.append(np.append(np.array([int(angle),I_ratio]),gauss_fit_to_spectrum(bins,spectrum,x1,x2, angle)))


In [ ]:
#%matplotlib qt
plt.rcParams.update({'font.size': 20})
plt.rcParams["font.family"] = "serif"

In [ ]:
angle='50'
normalized_data = am_al_dict[angle]['data']/int(am_al_dict[angle]['time']) #calibrations[angle]['data']/np.sum(calibrations[angle]['data'])
normalized_bg = background/840 #cesium_cs_pb_dict[angle]['data']/np.sum(cesium_cs_pb_dict[angle]['data'])

# Plot data and background with correct energy scaling
plt.figure(figsize = (10,5))
plt.plot(bins*m+b,normalized_data,label='Scaled Spectra',alpha=0.7)
plt.plot(bins*m+b,normalized_bg,label='Scaled Background',alpha=0.7)
plt.legend()

# Fit gaussian to peak and plot peak
x1 = am_al_dict[angle]['x1']
x2 = am_al_dict[angle]['x2']
scale = np.sum(am_al_dict[angle]['data'])
spectrum = (normalized_data - normalized_bg)
pb_data.append(np.append(np.array([int(angle),int(am_al_dict[angle]['time'])]),gauss_fit_to_spectrum(bins,spectrum,x1,x2, angle)))
plt.tight_layout()

In [ ]:
plt.plot(bins*m+b, normal_gauss(input_x, A, x0, sigma), '--r', label='fit')
plt.legend()
plt.title('Gaussian fit ' + angle + ' deg')
plt.xlabel('Energy Levels (KeV)')
plt.ylabel('Counts')
plt.show()

In [ ]:
# Get fit parameters
angles_am_al, times_am_al, shifts_am_al, As_am_al, sigmas_am_al, shifts_err_am_al, As_err_am_al, sigmas_err_am_al = np.transpose(np.array(al_data))
# Area under the gaussian fit
# intensity_am_al = [np.sum(gauss(bins,As_am_al[i], shifts_am_al[i], sigmas_am_al[i])) for i in range(len(angles_am_al))]

In [ ]:
As_am_al

In [ ]:

plt.figure(figsize=(10,5))
plt.plot(angles_am_al,shifts_am_al,'o',label='Aluminum')
plt.plot(angles_am_pb,shifts_am_pb,'o',label='Lead')
plt.xlabel('Angles (degrees)')
plt.ylabel('Peak Energy (keV)')
plt.title('Americium Source Compton Scattering')
plt.legend()
plt.tight_layout()


#### Compare compton shifts to theoretical values

In [ ]:
#%matplotlib qt
plt.rcParams.update({'font.size': 20})
plt.rcParams["font.family"] = "serif"

In [ ]:
mass_of_electron = 9.10938356e-31 #kg
speed_of_light = 299792458 #m/s
E0 = 59.5 #keV
keV = 1000 * 1.602176634e-19 #kg m^2/s^2 = J

thetas = np.linspace(0,180,180)
calculated_shifts = 1 / (E0*keV) + 1 / (mass_of_electron *  speed_of_light ** 2) * (1-np.cos(np.radians(thetas)))

In [ ]:
# Fit compton shifts, slope should correspond to 1 / mc^2, where m is electron mass
m_am_al,b_am_al = np.polyfit(1-np.cos(np.radians(angles_am_al)), 1/(shifts_am_al), 1)
thetas = np.linspace(0,180,180)

In [ ]:
shifts_err_am_al/shifts_am_al

In [ ]:
# Plot compton shifts

plt.figure(figsize=(10,5))
plt.errorbar(1-np.cos(np.radians(angles_am_al)), 1/(shifts_am_al),yerr=(shifts_err_am_al/shifts_am_al**2),fmt='o',markersize = 10, label = 'Data')
plt.plot(1-np.cos(np.radians(thetas)), calculated_shifts * keV, label = 'Theoretical Values')
# plt.plot(1-np.cos(np.radians(thetas)), calculated_shifts_adjusted * keV, label = 'Theoretical Values Adjusted')
plt.plot(1-np.cos(np.radians(thetas)), (1-np.cos(np.radians(thetas)))*m_am_al+b_am_al, '--', label = 'Data Fit')
plt.xlabel(r'1-cos($\theta$)')
plt.ylabel(r'1/E (keV$^{-1}$)')
plt.annotate('Fitted Slope = %4f keV$^{-1}$'%(m_am_al),(0.75,0.017))
plt.title('Americium Source Aluminum Scattering Target')
plt.legend()
plt.tight_layout()


# Lead

In [ ]:
files = glob.glob('Americium_Lead/*.Spe')
am_pb_dict = {}

%matplotlib inline
for f in files:
    angle = f.split('AmericiumPb')[1].split('degrees')[0]
    time = f.split('volts')[1].split('sec')[0]
    data = np.loadtxt(f,skiprows=12, max_rows=1023)
    entry = {'filename':f, 'time':time,'data':data}
    am_pb_dict[angle] = entry

In [ ]:
am_pb_dict['50']['x1'] = 900
am_pb_dict['50']['x2'] = 935

am_pb_dict['70']['x1'] = 850
am_pb_dict['70']['x2'] = 935

am_pb_dict['90']['x1'] = 800
am_pb_dict['90']['x2'] = 920

am_pb_dict['110']['x1'] = 750
am_pb_dict['110']['x2'] = 900

am_pb_dict['130']['x1'] = 750
am_pb_dict['130']['x2'] = 850

am_pb_dict['150']['x1'] = 750
am_pb_dict['150']['x2'] = 850

# am_al_dict['150']['x1'] = 650
# am_al_dict['150']['x2'] = 800

In [ ]:
bins = np.linspace(1,1023,1023)
plt.rcParams['figure.figsize'] = [10, 5]
pb_data = []

%matplotlib inline
for angle in ['50','70','90','110','130','150']:
    entry = am_pb_dict[angle]
    plt.figure(figsize=(10,5))
#     plt.plot(entry['data'], label = angle)
    x1 = entry['x1']
    x2 = entry['x2']
    bg_level = np.mean(entry['data'][400:600])/int(entry['time'])
    spectrum = entry['data']/int(entry['time'])-bg_level
#     bg_level = np.mean(background[x1:x2])
    plt.axhline(bg_level,linestyle='dashed')
#     bg_level = np.mean(background[x1:x2])
    pb_data.append(np.append(np.array([int(angle),int(am_pb_dict[angle]['time'])]),gauss_fit_to_spectrum(bins,spectrum,x1,x2, angle)))
#     plt.plot(bins,entry['data']-background * (int(entry['time'])/120), label = angle + ' deg')
#     plt.title(angle + ' deg')

In [ ]:
angles_am_pb, times_am_pb, shifts_am_pb, As_am_pb, sigmas_am_pb, shifts_err_am_pb, As_err_am_pb, sigmas_err_am_pb = np.transpose(np.array(pb_data))


In [ ]:
plt.figure()
plt.plot(angles_am_pb,As_am_pb,'o')
plt.xlabel('Angles')
plt.ylabel('Cross Section')
plt.title('Cesium Source Lead Scattering Target')

#### Compare compton shifts to theoretical values

In [ ]:
%matplotlib qt
plt.rcParams.update({'font.size': 20})
plt.rcParams["font.family"] = "serif"

In [ ]:
# Constants
mass_of_electron = 9.10938356e-31 #kg
speed_of_light = 299792458 #m/s
E0 = 59.5 #keV
keV = 1000 * 1.602176634e-19 #kg m^2/s^2 = J

thetas = np.linspace(0,180,180)
calculated_shifts = 1 / (E0*keV) + 1 / (mass_of_electron *  speed_of_light ** 2) * (1-np.cos(np.radians(thetas)))
# Fit compton shifts, slope should correspond to 1 / mc^2, where m is electron mass
m_am_pb,b_am_pb = np.polyfit(1-np.cos(np.radians(angles_am_pb)), 1/(shifts_am_pb), 1)
thetas = np.linspace(0,180,180)

plt.figure(figsize=(10,5))
plt.errorbar(1-np.cos(np.radians(angles_am_pb)), 1/(shifts_am_pb),yerr=shifts_err_am_pb/shifts_am_pb**2,fmt='o',markersize = 10, label = 'Data')
plt.plot(1-np.cos(np.radians(thetas)), calculated_shifts * keV, label = 'Theoretical Values')
# plt.plot(1-np.cos(np.radians(thetas)), calculated_shifts_adjusted * keV, label = 'Theoretical Values Adjusted')
plt.plot(1-np.cos(np.radians(thetas)), (1-np.cos(np.radians(thetas)))*m_am_pb+b_am_pb, '--', label = 'Data Fit')
plt.xlabel(r'1-cos($\theta$)')
plt.ylabel(r'1/E (keV$^{-1}$)')
plt.annotate('Fitted Slope = %4f keV$^{-1}$'%(m_am_pb),(0.75,0.017))
plt.title('Americium Source Lead Scattering Target')
plt.legend()
plt.tight_layout()

In [ ]:
plt.plot(thetas, 1/calculated_shifts, label = 'Theoretical Values')

In [ ]:
%matplotlib qt
plt.figure(figsize=(10,5))
plt.errorbar(angles_am_al, shifts_am_al,yerr=shifts_err_am_al,fmt='o', markersize = 10, label='Aluminum',alpha = 0.7)
plt.errorbar(angles_am_pb, shifts_am_pb,yerr=shifts_err_am_pb,fmt='o', markersize = 10, label='Lead',alpha=0.7)
plt.xlabel('Angles (radians)')
plt.ylabel('Scattered Photon Energy (keV)')
plt.title('Americium Scattering')
plt.legend()
plt.tight_layout()

In [ ]:
f, axes = plt.subplots(6, 1, sharex=True, figsize=(10,12))
axes[0].set_title('Americium 241 Source Scattered on Aluminum Target')
# ax7.set_xlabel('Energies (keV)')
# Make common axis labels
f.text(0.5, 0.04, 'Energies (keV)', va='center', ha='center')
f.text(0.02, 0.5, 'Intensity (counts/s)', va='center', ha='center', rotation='vertical')
angles =['30','50','70','90','110','130']
for i in range(len(angles)):
    axes[i].plot(bins*m+b,am_al_dict[angles[i]]['data']/int(am_al_dict[angles[i]]['time']))
    axes[i].annotate(angles[i]+' deg', (0.3,0.5), xycoords='axes fraction')

In [ ]:
f, axes = plt.subplots(6, 1, sharex=True, figsize=(10,12))
axes[0].set_title('Americium 241 Source Scattered on Lead Target')
# ax7.set_xlabel('Energies (keV)')
# Make common axis labels
f.text(0.5, 0.04, 'Energies (keV)', va='center', ha='center')
f.text(0.02, 0.5, 'Intensity (counts/s)', va='center', ha='center', rotation='vertical')
angles = ['50','70','90','110','130','150']
for i in range(len(angles)):
    axes[i].plot(bins*m+b, am_pb_dict[angles[i]]['data']/int(am_pb_dict[angles[i]]['time']))
    axes[i].annotate(angles[i]+' deg', (0.3,0.5), xycoords='axes fraction')

## Cross Section

In [ ]:
re = 2.818e-13 #cm classical radiums of the electron
keV_g = 1.602176634e-9 #g cm^2/s^2 = erg
speed_of_light_cm = 29979245800 #cm/s
mass_of_electron = 9.10938356e-31 #kg
mass_of_electron_g = 9.10938356e-28 #kg
# plank = 6.62607004e-34
compton_wavelength = 2.42631023867e-12 #m
compton_wavelength_cm = 2.42631023867e-10 #cm


r = 25.5 #cm, distance from target to detector
h_target = 7.7 #cm
d_target = 1.9 #cm
crystal_area = 5.08 * 5.08 #cm**2

N0 = 6e23 #Avogadro's number
# What is the Cs's Currie?
I0 = 3.7e10 * 3.69e-3 / (4 * np.pi * r**2) # 1/(cm^2 s)

A_al = 26.9 #atomic weight of aluminum
A_pb = 207.2 #atomic weight of Lead
Z_al = 13 #atomic number of aluminum
Z_pb = 82 #atomic number of Lead

rho_al = 2.7 #g/cm^3
rho_pb = 11.35 #g/cm^3

N_al = np.pi * ( d_target / 2) **2 * h_target * rho_al * N0 / A_al * Z_al
N_pb = np.pi * ( d_target / 2) **2 * h_target * rho_pb * N0 / A_pb * Z_pb


In [ ]:
def Klein_Nishina(E0, theta):
    a = E0 * keV_g / (mass_of_electron_g *  speed_of_light_cm ** 2) # should be 1.29 for 662 keV
    cos = np.cos(np.radians(theta))
    return (re ** 2) * ((1+cos**2) / 2) * ( 1 / (1 + a * (1-cos)) **2 ) * (1 + a**2*(1-cos)**2 / (1+cos**2) / (1 + a*(1-cos)))

def Klein_Nishina_Etheta(E0, E_theta, theta):
    PE = E_theta/E0
    return 1 / 2 * re ** 2 * PE ** 2 * (  PE + 1/PE - np.sin(np.radians(theta)) ** 2)

def Compton_Shift(E0, theta):
    return 1 / (E0*keV) + 1 / (mass_of_electron *  speed_of_light ** 2) * (1-np.cos(np.radians(thetas)))
 

In [ ]:
plt.plot(thetas, Klein_Nishina(59.5, thetas))
plt.plot(thetas, Klein_Nishina(662, thetas))


In [ ]:
662 * keV_g / (mass_of_electron_g *  speed_of_light_cm ** 2)

In [ ]:
1/ (mass_of_electron *  speed_of_light ** 2) * keV

In [ ]:
Klein_Nishina(662, 5)

#### Theoretical values using Klein-Nishina

In [ ]:
%matplotlib qt
plt.rcParams.update({'font.size': 20})
plt.rcParams["font.family"] = "serif"

In [ ]:
thetas = np.linspace(0,180,180)
# E_thetas = Compton_Energy(59.5, thetas)
diff_cross_secs = Klein_Nishina(59.5, thetas)
plt.plot(thetas,diff_cross_secs)

In [ ]:
plt.plot(angles_am_al,shifts_am_al,'o')

In [ ]:
# plt.figure(figsize=(20,10))
plt.figure(figsize=(12,6))
plt.plot(angles_am_al, Klein_Nishina_Etheta(59.5, shifts_am_al , angles_am_al),'o',markersize = 10, alpha=0.5, label = 'Aluminum Target')
plt.plot(angles_am_pb, Klein_Nishina_Etheta(59.5, shifts_am_pb , angles_am_pb),'o',markersize = 10, alpha=0.5,label = 'Lead Target' )
plt.plot(thetas,diff_cross_secs)
plt.title('Differential Cross Section of Scattered Americium')
plt.xlabel('Angles (radians)')
plt.ylabel('Differential Cross Section ($cm^2/sr$)')
plt.legend()
plt.tight_layout()
# plt.plot(thetas, 1 / E_thetas,'o')

#### Calculated Values

In [ ]:
np.array(As_am_al) / ( (crystal_area / r ** 2) * N_al * I0)

In [ ]:
Klein_Nishina(59.5, 130)

In [ ]:
%matplotlib inline

plt.figure()
al_const = ((crystal_area / r ** 2) * N_al * I0)
pb_const = ((crystal_area / r ** 2) * N_pb * I0)
plt.errorbar(angles_am_al,np.array(As_am_al) / al_const ,yerr = As_err_am_al/al_const,fmt='o')
# plt.errorbar(angles_am_pb,np.array(As_am_pb) / pb_const,yerr = As_err_am_pb/pb_const ,fmt='o')
plt.plot(thetas, diff_cross_secs)
plt.xlabel('Angles')
plt.ylabel('Cross Section')
plt.title('Cesium Source Aluminum Scattering Target')

In [ ]:
np.mean(np.array(As_am_al) / Klein_Nishina_Etheta(59.5, shifts_am_al , angles_am_al))

In [ ]:
%matplotlib qt

plt.figure(figsize=(10,5))
# plt.plot(angles_cs_al,np.array(As_cs_al)/As_cs_al[0] / ( (crystal_area / r ** 2) * N_al * I0),'o',label='Measured')
plt.errorbar(angles_am_al,np.array(As_am_al),yerr=As_err_am_al,fmt='o',label='Measured')
plt.plot(angles_am_al, Klein_Nishina_Etheta(59.5, shifts_am_al , angles_am_al)*7.7e23,'o',markersize = 10, alpha=0.5, label = 'Klein-Nishina')
plt.plot(thetas, diff_cross_secs*7.7e23)

plt.xlabel('Angles (degrees)')
plt.ylabel('Cross Section (cm^2/sr)')
plt.title('Americium Source Aluminum Scattering Target')
plt.legend()



In [ ]:
%matplotlib qt

plt.figure(figsize=(12,6))
plt.errorbar(angles_am_al,np.array(As_am_al) / al_const ,yerr = As_err_am_al/al_const ,fmt='o',label='Measured')
plt.xlabel('Angles (degrees)')
plt.ylabel('Cross Section (cm^2/sr)')
plt.title('Americium Source Aluminum Scattering Target')
plt.legend()
plt.tight_layout()

# plt.figure(figsize=(10,5))
# plt.plot(angles_am_al, Klein_Nishina_Etheta(59.5, shifts_am_al , angles_am_al),'o',markersize = 10, alpha=0.5, label = 'Klein-Nishina')
# plt.xlabel('Angles (degrees)')
# plt.ylabel('Cross Section (cm^2/sr)')
# plt.title('Americium Source Aluminum Scattering Target')
# plt.legend()
# plt.tight_layout()

plt.figure(figsize=(12,6))
plt.errorbar(angles_am_pb,np.array(As_am_pb) / pb_const,yerr = As_err_am_pb/pb_const ,fmt='o',label='Measured')
# plt.plot(thetas, diff_cross_secs)
plt.xlabel('Angles (degrees)')
plt.ylabel('Cross Section (cm^2/sr)')
plt.title('Americium Source Lead Scattering Target')
plt.legend()
plt.tight_layout()

# plt.figure(figsize=(10,5))
# plt.plot(angles_am_pb, Klein_Nishina_Etheta(59.5, shifts_am_pb , angles_am_pb),'o',markersize = 10, alpha=0.5,label = 'Klein-Nishina' )
# # plt.plot(thetas, diff_cross_secs)
# plt.xlabel('Angles (degrees)')
# plt.ylabel('Cross Section (cm^2/sr)')
# plt.title('Americium Source Lead Scattering Target')
# plt.legend()
# plt.tight_layout()